## Some notes on Newman's 5.19 - Diffraction Gratings

$w =$ the distance along the diffraction grating itself *note that you will need a separate parameter for the screen width*

$u =$ the distance up and down the diffraction grating itself.  You integrate along it to find all of the light
coming from the diffraction grating.  

$x =$ distance along the SCREEN where the pattern is created (like the one in Newman) – 

$q(u) =$ geometrical weight that takes into account superposition of light rays.  

You could derive $q(u)$ with calculus (if someone hasn’t given it to you).  Basically if you have a grating, you treat each opening like an individual point of light and calculate the pattern that will form on a distant screen – it makes a pattern since geometrically different light rays have different lengths, so they arrive at different phases – that may add or cancel depending on the length of each ray.  I write about this in detail in Lab 4 solutions if you want to have a peak.

While it is fine to accept the value given as $q(u) = sin^2(\alpha u)$, you’re correct that doesn’t help you answer (a) where you want to find the spacing of the grating.  For this, you can go look at an optics book, or you can be sneaky and back it out using a little logic/calculus/magic: 

* Notice that the function q(u) has zeros, $z$, find an expression for $z$ in terms of $\alpha$ (use trig here!!!  There will be a $\pi$ in your answer).  
* These $z$ values are special cases of u that have the same periodicity as the geometrical pattern $q(u)$.  So find the spacing between two z values, and you have found the slit separation
* Since the spacing is a geometrical quantity that a machinist might make, you can take the spacing to be the independent variable and calculate $\alpha$ from it.

So you have to loop over all your $x-$values, and for each $x-$value add up all of the light coming from the entire diffraction grating.  That is the picture Newman has given you – it shows all of the light hitting one particular $x-$value, but imagine the rest of the x-values having a similar picture, arising from a different set of light rays emanating from the diffraction grating.
